# reading our Farsi data

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving digikala_dataset.txt to digikala_dataset.txt


In [0]:
file1 = open("digikala_dataset.txt","r") 
x = file1.readlines()
data = []
data_labels = []
for item in x:
  t = item.split('\t')
  data.append(t[1])
  data_labels.append(t[0])

In [0]:
data_labels = list(map(int, data_labels))

In [0]:
one =0
zero = 0
neutral = 0
pos_data = []
neu_data = []
neg_data = []
i = 0
for i,item in enumerate(data_labels):
  if item == 1:
    one += 1
    pos_data.append(data[i])
  elif item == -1:
    zero += 1
    neg_data.append(data[i])
  elif item == 0:
    neutral +=1
    neu_data.append(data[i])
    
print("number of positives is : " + str(one/(one + zero + neutral)))
print(one)
print("number of negetives is : " + str(zero/(one + zero + neutral)))
print(zero)
print("number of unlabeled is : " + str(neutral/(one + zero + neutral)))
print(neutral)
print(one + zero + neutral)

number of positives is : 0.22832621873770303
45839
number of negetives is : 0.020900473697580704
4196
number of unlabeled is : 0.7507733075647163
150726
200761


In [0]:
from random import sample 
data_labels = []
data = []
sampled_pos = sample(pos_data,len(neg_data))
data = sampled_pos + neg_data
for i in range(len(sampled_pos)):
  data_labels.append(1)
  
for i in range(len(neg_data)):
  data_labels.append(0)

In [0]:
from random import sample 
data_labels = []
data = []
data = pos_data + neg_data
for i in range(len(pos_data)):
  data_labels.append(1)
  
for i in range(len(neg_data)):
  data_labels.append(0)
  

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, data_labels, test_size=0.1, random_state=42)
print(len(X_train))
print(len(y_test))

45031
5004


In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [0]:
!pip install bert-tensorflow

In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'OUTPUT_DIR_NAME'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: OUTPUT_DIR_NAME *****


In [0]:
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences. This doesn't apply to our task, so we can leave `text_b` blank.
- `label` label

In [0]:
train_InputExamples = []
test_InputExamples = []
train_labels = []
test_labels = []
pos = 0
neg = 0

for item in zip(X_train,y_train):
    if(item[1] == 1):
      train_InputExamples.append(bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = item[0], 
                                                                   text_b = None, 
                                                                   label = 1))
      train_labels.append(1)
    
      
    if(item[1] == 0):
      train_InputExamples.append(bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = item[0], 
                                                                   text_b = None, 
                                                                   label = 0))
      train_labels.append(0)
      
  
  
for item in zip(X_test,y_test):
    if(item[1] == 1):
      test_InputExamples.append(bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = item[0], 
                                                                   text_b = None, 
                                                                   label = 1))
      test_labels.append(1)
      pos = pos + 1
    
    if(item[1] == 0):
      test_InputExamples.append(bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = item[0], 
                                                                   text_b = None, 
                                                                   label = 0))
      train_labels.append(0)
      neg = neg + 1
print(pos)
print(neg)
print(len(test_InputExamples))
      

4581
423
5004


Next, we need to preprocess our data so that it matches the data BERT was trained on.


1. Lowercase our text
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens 
6. Append "index" and "segment" tokens to each input 




In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
BERT_MODEL_HUB_CASE = "https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB_CASE)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=False)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0524 14:07:59.488316 139622375171968 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
tokenizer.tokenize("به نظرم انتخاب خوبی نیست")

['به', 'نظر', '##م', 'انتخاب', 'خوبی', 'نیست']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples,label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 45031


I0524 14:43:08.606564 139622375171968 run_classifier.py:774] Writing example 0 of 45031


INFO:tensorflow:*** Example ***


I0524 14:43:08.610391 139622375171968 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0524 14:44:00.077051 139622375171968 run_classifier.py:466] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0524 14:44:00.079798 139622375171968 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


I0524 14:44:00.084060 139622375171968 run_classifier.py:468] label: 1 (id = 1)


#Creating a model

Now that we've prepared our data. `create_model` does just this below. First, it loads the BERT tf hub module  to extract the computation graph. Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a review is positive or negative).

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB_CASE,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

   
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


evaluation, and prediction.

In [0]:
# using the passed parameters for num_labels, learning_rate.
def eval_confusion_matrix(labels, predictions):
    with tf.variable_scope("eval_confusion_matrix"):
        con_matrix = tf.confusion_matrix(labels=labels, predictions=predictions, num_classes=2)

        con_matrix_sum = tf.Variable(tf.zeros(shape=(2,2), dtype=tf.int32),
                                            trainable=False,
                                            name="confusion_matrix_result",
                                            collections=[tf.GraphKeys.LOCAL_VARIABLES])


        update_op = tf.assign_add(con_matrix_sum, con_matrix)

        return tf.convert_to_tensor(con_matrix_sum), update_op
      
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        con = eval_confusion_matrix(label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg,
            "confusion_martix": con
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 8.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
print(num_train_steps )

1888


In [0]:
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efbd17f13c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0524 14:08:12.017679 139622375171968 estimator.py:201] Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efbd17f13c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [0]:
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
print(num_train_steps)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
1888
INFO:tensorflow:Calling model_fn.


I0524 14:08:19.555268 139622375171968 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0524 14:08:22.846434 139622375171968 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 1888 into OUTPUT_DIR_NAME/model.ckpt.


I0524 14:39:45.414206 139622375171968 basic_session_run_hooks.py:594] Saving checkpoints for 1888 into OUTPUT_DIR_NAME/model.ckpt.


INFO:tensorflow:Loss for final step: 0.00018463293.


I0524 14:40:01.598744 139622375171968 estimator.py:359] Loss for final step: 0.00018463293.


Training took time  0:31:46.287634


Now let's use our test data to see how well our model did:

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)
print(len(test_features))

5004


In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0524 14:44:22.235211 139622375171968 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0524 14:44:25.911881 139622375171968 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Summary for np.ndarray is not visible in Tensorboard by default. Consider using a Tensorboard plugin for visualization (see https://github.com/tensorflow/tensorboard-plugin-example/blob/master/README.md for more information).


I0524 14:45:28.558697 139622375171968 estimator.py:2012] Summary for np.ndarray is not visible in Tensorboard by default. Consider using a Tensorboard plugin for visualization (see https://github.com/tensorflow/tensorboard-plugin-example/blob/master/README.md for more information).


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1888: OUTPUT_DIR_NAME/model.ckpt-1888


I0524 14:45:28.566046 139622375171968 estimator.py:2039] Saving 'checkpoint_path' summary for global step 1888: OUTPUT_DIR_NAME/model.ckpt-1888


{'confusion_martix': array([[ 415,    8],
        [ 849, 3732]], dtype=int32),
 'eval_accuracy': 0.828737,
 'false_negatives': 849.0,
 'false_positives': 8.0,
 'global_step': 1888,
 'loss': 1.1704502,
 'precision': 0.99786097,
 'recall': 0.8146693,
 'true_negatives': 415.0,
 'true_positives': 3732.0}

make predictions on new sentences:

In [0]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = [
  "ساعت خوبی بود از نظر ظاهر و سنسوراش خوب بود_x000D_ولی یه بدی بزرگی که داره بعد 3 ماه بندش خشک گرفت و پاره شد بندشم هیچ جا حتی دیجی نداشتx000D_عملا دیگه قابل استفاده نبود_x000D_ به نظرم با توجه به نظرات بقیه هم که گفتن پاره میشه تا وقتی که بند یدکی نداره نخرید پولتون میریزید دور",
  "ارزش خرید نداره. هزینه بیشتر بکنید و گزینه های بهتر خریداری کنین.",
    "حتما گوشی واقعن خوبه خیلی خوبه a7 2018 رو بخرین واقعا برای اونایی که گوشی رو دست نگرفتنو نظر بد میدن همش اشتباه اصلا داغ نمیکنه اصلا هنگ نمیکنه و دوربینشم فوق العاده",
    "مشکل شارژ باطری داره. شارژر درست وارد گوشی نمیشه و در حالت عادی هم سریع باطری خالی میکنه",
    "این گوشی شارژش خیلی بده و اصلا نگه نمی داره"
]

In [0]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 5


I0524 14:06:23.110162 139622375171968 run_classifier.py:774] Writing example 0 of 5


INFO:tensorflow:*** Example ***


I0524 14:06:23.119384 139622375171968 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0524 14:06:28.174926 139622375171968 saver.py:1270] Restoring parameters from OUTPUT_DIR_NAME/model.ckpt-3776


INFO:tensorflow:Running local_init_op.


I0524 14:06:29.000250 139622375171968 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0524 14:06:29.093328 139622375171968 session_manager.py:493] Done running local_init_op.


We have a sentiment classifier now!

In [0]:
predictions

[('ساعت خوبی بود از نظر ظاهر و سنسوراش خوب بود_x000D_ولی یه بدی بزرگی که داره بعد 3 ماه بندش خشک گرفت و پاره شد بندشم هیچ جا حتی دیجی نداشتx000D_عملا دیگه قابل استفاده نبود_x000D_ به نظرم با توجه به نظرات بقیه هم که گفتن پاره میشه تا وقتی که بند یدکی نداره نخرید پولتون میریزید دور',
  array([-4.3748852e-05, -1.0036139e+01], dtype=float32),
  'Negative'),
 ('ارزش خرید نداره. هزینه بیشتر بکنید و گزینه های بهتر خریداری کنین.',
  array([-3.2901222e-05, -1.0321674e+01], dtype=float32),
  'Negative'),
 ('حتما گوشی واقعن خوبه خیلی خوبه a7 2018 رو بخرین واقعا برای اونایی که گوشی رو دست نگرفتنو نظر بد میدن همش اشتباه اصلا داغ نمیکنه اصلا هنگ نمیکنه و دوربینشم فوق العاده',
  array([-1.7915551e-04, -8.6270285e+00], dtype=float32),
  'Negative'),
 ('مشکل شارژ باطری داره. شارژر درست وارد گوشی نمیشه و در حالت عادی هم سریع باطری خالی میکنه',
  array([-6.901980e-05, -9.581642e+00], dtype=float32),
  'Negative'),
 ('این گوشی شارژش خیلی بده و اصلا نگه نمی داره',
  array([-2.4556812e-05, -1.0612304e+01],